In [16]:
from subprocess import run
from pathlib import Path
from dataclasses import dataclass
from typing import Optional


def bazel(*args) -> str:
    proc = run(["bazel", *args], check=True, capture_output=True)

    return proc.stdout.decode()


workspace = Path(bazel("info", "workspace").splitlines()[0])


@dataclass
class TimeEntry:
    operation: str
    clockcycles: int

    @staticmethod
    def parse(line: str) -> Optional["TimeEntry"]:
        if len(parts := line.split("\t")) != 3:
            return None

        start = int(parts[1].removeprefix("0x"), base=16)
        end = int(parts[2].removeprefix("0x"), base=16)

        return TimeEntry(parts[0], end - start)


output = bazel("run", "//experiments/bitrot:metrics-sim65", "--run_under=@cc65//:sim65")

[
    time_entry
    for line in output.splitlines()
    if (time_entry := TimeEntry.parse(line)) is not None
]

CalledProcessError: Command '['bazel', 'info', 'workspace']' returned non-zero exit status 37.

In [11]:
output = bazel("build", "//experiments/bitrot:c64")
output = bazel("cquery", "--output=files", "//experiments/bitrot:c64")

files = [workspace / file for file in output.splitlines()]

s = files[0].stat()

s

os.stat_result(st_mode=33133, st_ino=2090593385, st_dev=16777231, st_nlink=1, st_uid=501, st_gid=20, st_size=1461, st_atime=1769630506, st_mtime=1769630506, st_ctime=1769630506)